In [1]:
from __future__ import print_function
from openpathsampling.experimental.storage import Storage, ops_class_info, ops_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
import numpy as np
from openpathsampling.experimental.simstore.serialization_helpers import get_uuid, set_uuid

import openpathsampling as paths

import collections

from openpathsampling.experimental.simstore.storable_functions import (
    StorableFunction, StorageFunctionHandler
)

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

## Playing around with a storable function

In [3]:
def get_x(snap):
    from time import sleep
    sleep(2)
    return snap.xyz[0][0]

In [4]:
cv = StorableFunction(get_x, result_type='float')

In [5]:
import openpathsampling.engines.toy as toys
snap = toys.Snapshot(coordinates=np.array([[3.0, 0.0]]))

In [6]:
%%time
cv(snap)

2020-10-20 17:04:16,609 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:16,611 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:16,614 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
CPU times: user 4.83 ms, sys: 1.98 ms, total: 6.81 ms
Wall time: 2.01 s


3.0

In [7]:
%%time
cv(snap)

2020-10-20 17:04:18,636 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
CPU times: user 1.5 ms, sys: 611 µs, total: 2.11 ms
Wall time: 1.96 ms


3.0

## Manually registering storable function with SQL backend

In [8]:
backend = SQLStorageBackend("test.sql", mode='w', echo=False)
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info,
)

2020-10-20 17:04:18,681 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-10-20 17:04:18,686 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-10-20 17:04:18,720 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-10-20 17:04:18,757 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-10-20 17:04:18,772 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-10-20 17:04:18,808 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-10-20 17:04:18,820 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-10-20 17:04:18,842 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-10-20 17:04:18,858 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [9]:
uuid = get_uuid(cv)

In [10]:
_ = cv(snap)
cv.local_cache.result_dict

2020-10-20 17:04:18,992 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>


{'176170959556776523988076590496099598352': 3.0}

In [11]:
backend.register_storable_function(table_name=uuid, result_type=cv.result_type)

2020-10-20 17:04:19,005 - openpathsampling.experimental.simstore.sql_backend - INFO - Registering storable function: UUID: 176170959556776523988076590496099598348 (float)


In [12]:
backend.add_storable_function_results(table_name=uuid,
                                      result_dict=cv.local_cache.result_dict)

2020-10-20 17:04:19,029 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [13]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2020-10-20 17:04:19,043 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'176170959556776523988076590496099598352': 3.0}

In [14]:
backend.load_storable_function_table(get_uuid(cv))

{'176170959556776523988076590496099598352': 3.0}

## New backend; use storable func with storage

This will be roughly the full integration.

In [15]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-10-20 17:04:19,078 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-10-20 17:04:19,082 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-10-20 17:04:19,091 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-10-20 17:04:19,101 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-10-20 17:04:19,145 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-10-20 17:04:19,158 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-10-20 17:04:19,169 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-10-20 17:04:19,180 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-10-20 17:04:19,188 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [16]:
backend.identifier

('sqlite:///test.sql', 'w')

In [17]:
storage.save(cv)

2020-10-20 17:04:19,291 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:19,296 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:19,300 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:19,303 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:19,304 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-10-20 17:04:19,307 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-10-20 17:04:19,309 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-10-20 17:04:19,310 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-10-20 17:04:19,314 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:19,315 - openp

In [18]:
list(storage.simulation_objects)

[]

In [19]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'samples', 'sample_sets', 'trajectories', 'move_changes', 'steps', 'details', 'storable_functions', 'simulation_objects', 'storage_objects', '176170959556776523988076590496099598348'])

In [20]:
storage._sf_handler.functions[0].local_cache.result_dict

{'176170959556776523988076590496099598352': 3.0}

In [21]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2020-10-20 17:04:19,407 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'176170959556776523988076590496099598352': 3.0}

In [22]:
storage.close()

In [23]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-10-20 17:04:19,472 - openpathsampling.experimental.simstore.storage - INFO - Missing info from 0 dynamically-registered tables
2020-10-20 17:04:19,478 - openpathsampling.experimental.storage.ops_storage - INFO - Found 0 possible lookups
2020-10-20 17:04:19,478 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys([])
2020-10-20 17:04:19,480 - openpathsampling.experimental.simstore.storage - INFO - Successfully registered 0 missing tables
2020-10-20 17:04:19,485 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2020-10-20 17:04:19,485 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2020-10-20 17:04:19,486 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2020-10-20 17:04:19,487 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-10-20 17:04:19,489 

In [24]:
cv_reloaded = storage.load([get_uuid(cv)])[0]

2020-10-20 17:04:19,509 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2020-10-20 17:04:19,510 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 1 non-cached objects
2020-10-20 17:04:19,512 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:19,515 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:19,528 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-10-20 17:04:19,533 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 1 objects; creating 0 lazy proxies
2020-10-20 17:04:19,535 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2020-10-20 17:04:19,536 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2020-10-20 17:04:19,538 - openpathsampling.experimental.simstore.sql_backend - DEBUG 

In [25]:
cv_reloaded.local_cache.result_dict

{}

In [26]:
cv_reloaded.local_cache != cv.local_cache

True

In [27]:
%%time
cv_reloaded.local_cache.clear()
cv_reloaded(snap)

2020-10-20 17:04:19,572 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x104999470>>
2020-10-20 17:04:19,574 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x104999470>>
2020-10-20 17:04:19,578 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
CPU times: user 5.27 ms, sys: 1.71 ms, total: 6.99 ms
Wall time: 7.45 ms


3.0

In [28]:
# TODO: tests need a test of a bad load as well (missing should show up)

In [29]:
cv_reloaded.local_cache.clear()
cv_reloaded.preload_cache()

In [30]:
%%time
cv_reloaded(snap)

2020-10-20 17:04:19,607 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x104999470>>
CPU times: user 1.58 ms, sys: 1.27 ms, total: 2.85 ms
Wall time: 3.06 ms


3.0

## Store/load multiple values at once

This checks that the backend is able to load multiple UUIDs at once

In [31]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-10-20 17:04:19,627 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-10-20 17:04:19,629 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-10-20 17:04:19,643 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-10-20 17:04:19,661 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-10-20 17:04:19,667 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-10-20 17:04:19,683 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-10-20 17:04:19,689 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-10-20 17:04:19,699 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-10-20 17:04:19,707 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [32]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))

In [33]:
%%time
cv.local_cache.clear()
cv._handler = None
cv([snap, new_snap])

2020-10-20 17:04:19,780 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:19,783 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:19,786 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
CPU times: user 5.91 ms, sys: 2.05 ms, total: 7.96 ms
Wall time: 4.01 s


[3.0, 2.0]

In [34]:
%%time
# get using cache
cv([snap, new_snap])

2020-10-20 17:04:23,803 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
CPU times: user 1.59 ms, sys: 798 µs, total: 2.39 ms
Wall time: 2.74 ms


[3.0, 2.0]

In [35]:
storage.save(cv)

2020-10-20 17:04:23,817 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:23,819 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:23,823 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:23,827 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:23,831 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-10-20 17:04:23,835 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-10-20 17:04:23,842 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-10-20 17:04:23,845 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-10-20 17:04:23,868 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:23,870 - openp

In [36]:
%%time
# get using storage
cv.local_cache.clear()
cv([snap, new_snap])

2020-10-20 17:04:23,956 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:23,961 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x1144f0240>>
2020-10-20 17:04:23,968 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 2 UUIDs
CPU times: user 8.73 ms, sys: 2.88 ms, total: 11.6 ms
Wall time: 16.8 ms


[3.0, 2.0]

## Save multiple times

The second time we save things, the `find_uuid` skips over the already-saved function. This means that we need another mechanism for storing mutable information (such as the `StorableFunctionResults`) to disk.

This is part of what the `StorableFunctionHandler` manages. Since the handler contains the function, it can store things to disk with an update.

In [37]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)
cv = StorableFunction(get_x, result_type='float')
vol = paths.CVDefinedVolume(cv, lambda_min=0.0, lambda_max=1.0)
cv(snap)
storage.save(vol)

2020-10-20 17:04:23,992 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-10-20 17:04:23,994 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-10-20 17:04:24,010 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-10-20 17:04:24,016 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-10-20 17:04:24,039 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-10-20 17:04:24,045 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-10-20 17:04:24,054 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-10-20 17:04:24,062 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-10-20 17:04:24,072 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [38]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))
cv(new_snap)

2020-10-20 17:04:26,318 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11457ae10>>
2020-10-20 17:04:26,321 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11457ae10>>
2020-10-20 17:04:26,350 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:26,351 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11457ae10>>


2.0

In [39]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'176170959556776523988076590496099598352': 3.0}

In [40]:
storage.save(vol)

2020-10-20 17:04:28,379 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:28,383 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:28,385 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:28,393 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


In [41]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'176170959556776523988076590496099598352': 3.0}

In [42]:
storage.save_function_results()

2020-10-20 17:04:28,483 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [43]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'176170959556776523988076590496099598352': 3.0,
 '176170959556776523988076590496099598386': 2.0}

In [44]:
list(storage._sf_handler.canonical_functions.values())[0].local_cache.result_dict

{'176170959556776523988076590496099598352': 3.0,
 '176170959556776523988076590496099598386': 2.0}

## Mock of multiple computers creating and returning results

In [45]:
cv = StorableFunction(get_x, result_type='float')
cv(snap)

2020-10-20 17:04:28,529 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11465dc50>>
2020-10-20 17:04:28,531 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11465dc50>>
2020-10-20 17:04:28,533 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x11465dc50>>


3.0

### Computer 1

We mock the serialization and transfer over the network by doing a `to_dict`/`from_dict` cycle, which will create a new instance in memory with the same UUID.

In [46]:
cv_1 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_1, get_uuid(cv))

In [47]:
print("\n".join([repr(cv), repr(cv_1)]))

In [48]:
cv_1.local_cache.result_dict

{}

In [49]:
snap_1 = toys.Snapshot(coordinates=np.array([[4.0, 0.0]]))

In [50]:
%%time
cv_1(snap_1)

2020-10-20 17:04:30,608 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114549080>>
2020-10-20 17:04:30,612 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114549080>>
2020-10-20 17:04:30,613 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114549080>>
CPU times: user 4.61 ms, sys: 2.33 ms, total: 6.94 ms
Wall time: 2.01 s


4.0

In [51]:
cv_1.local_cache.result_dict

{'176170959556776523988076590496099598396': 4.0}

### Computer 2

In [52]:
cv_2 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_2, get_uuid(cv))

In [53]:
print("\n".join([repr(cv), repr(cv_1), repr(cv_2)]))

In [54]:
snap_2 = toys.Snapshot(coordinates=np.array([[5.0, 0.0]]))

In [55]:
cv_2(snap_2)

2020-10-20 17:04:32,686 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114699668>>
2020-10-20 17:04:32,692 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114699668>>
2020-10-20 17:04:32,700 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x114699668>>


5.0

In [56]:
cv_2.local_cache.result_dict

{'176170959556776523988076590496099598402': 5.0}

### Gather operation

Note that for the real thing, we'll need to ensure that the serializers/deserializers (be they from `dask` or custom) correctly recreate the objects.

Attention: The save command must be issued separately for each copy of the CV. This is because the CVs have the same UUID, and we use sets/dicts to combine repeats of the same UUID.

In [57]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2020-10-20 17:04:34,740 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2020-10-20 17:04:34,742 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2020-10-20 17:04:34,757 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2020-10-20 17:04:34,793 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2020-10-20 17:04:34,811 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2020-10-20 17:04:34,846 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2020-10-20 17:04:34,872 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table steps
2020-10-20 17:04:34,881 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table details
2020-10-20 17:04:34,949 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table s

In [58]:
storage.save([cv])

2020-10-20 17:04:35,076 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:35,078 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:35,079 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:35,083 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:35,084 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-10-20 17:04:35,087 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-10-20 17:04:35,088 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 2 UUIDs
2020-10-20 17:04:35,089 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 2 UUIDs
2020-10-20 17:04:35,093 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:35,095 - openp

In [59]:
storage.save([cv_1])

2020-10-20 17:04:35,133 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:35,134 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:35,135 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:35,139 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-10-20 17:04:35,140 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-10-20 17:04:35,141 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-10-20 17:04:35,142 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:35,143 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:35,147 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:35,151 - openp

In [60]:
storage.save([cv_2])

2020-10-20 17:04:35,177 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2020-10-20 17:04:35,183 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:35,184 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:35,188 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2020-10-20 17:04:35,190 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2020-10-20 17:04:35,191 - openpathsampling.experimental.simstore.storage - DEBUG - Checking if objects already exist in database
2020-10-20 17:04:35,192 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2020-10-20 17:04:35,193 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2020-10-20 17:04:35,196 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2020-10-20 17:04:35,197 - openp

In [61]:
cv_2.local_cache.parent

In [62]:
cv.local_cache.result_dict

{'176170959556776523988076590496099598352': 3.0,
 '176170959556776523988076590496099598396': 4.0,
 '176170959556776523988076590496099598402': 5.0}

In [63]:
storage._sf_handler.canonical_functions

{'176170959556776523988076590496099598388': <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x11465dc50>}

In [64]:
print("\n".join([str(getattr(obj, '__uuid__')) for obj in [cv, cv_reloaded, cv_1, cv_2]]))

176170959556776523988076590496099598388
176170959556776523988076590496099598348
176170959556776523988076590496099598388
176170959556776523988076590496099598388


In [65]:
results = storage._sf_handler.canonical_functions[get_uuid(cv)].local_cache.result_dict

In [66]:
assert len(results) == 3

In [67]:
results

{'176170959556776523988076590496099598352': 3.0,
 '176170959556776523988076590496099598396': 4.0,
 '176170959556776523988076590496099598402': 5.0}

In [68]:
# should be Dict[str, List[StorableFunction]]
# dict length 1; the single value in dict is list length 3
storage._sf_handler.all_functions

defaultdict(list,
            {'176170959556776523988076590496099598388': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x11465dc50>,
              <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x114699668>]})

In [69]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 3

In [70]:
storage._sf_handler.clear_non_canonical()

In [71]:
storage._sf_handler.all_functions

defaultdict(list,
            {'176170959556776523988076590496099598388': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x11465dc50>]})

In [72]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 1